In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from utils.paths import DATA_RAW_DIR, DATA_PROCESSED_DIR, MODELS_DIR

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.decomposition import LatentDirichletAllocation

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

import joblib

import re
import string

import nltk
from nltk.corpus import stopwords
import spacy



In [2]:
path_reviews = DATA_PROCESSED_DIR / "reviews_cleaned.csv"
print(Path(path_reviews).exists())

True


In [3]:
df_reviews = pd.read_csv(path_reviews, sep=",", encoding="utf-8")
df_reviews.head(10)

,lemmatized_reviews,stars
0,bueno ko pantallar menos mes recibir respuesta...,1
1,horrible comprar saber ingls informtico despus...,1
2,obligar comprar dos unidad llegar solo forma r...,1
3,entrar descalificar vendedor solo poder decir ...,1
4,llegar tarde co talla equivocado,1
5,jams lleg vendedor nunca contacto conmigo pesa...,1
6,paragua malo calidadda sensacin ir romper abr ...,1
7,devolver él ser triangular agarrir forma cmodo...,1
8,esperar despu protestar varios vez ver solucio...,1
9,defectuoso apariencia bien producto tercer uso...,1


In [4]:
df_reviews.isnull().sum()

lemmatized_reviews    1
stars                 0
dtype: int64

In [5]:
df_reviews.columns

Index(['lemmatized_reviews', 'stars'], dtype='object')

In [6]:
df_reviews.dropna(subset=["lemmatized_reviews"], inplace=True)

In [7]:
df_reviews.shape

(209999, 2)

In [8]:
df_reviews.duplicated().sum()

np.int64(3609)

In [9]:
df_reviews[df_reviews.duplicated()]

,lemmatized_reviews,stars
1964,rompi primero dar,1
2070,llegar an,1
2072,llegar producto,1
2288,producto llegar,1
2380,buen calidad,1
...,...,...
209893,mejor mejor,5
209918,bonito practico,5
209930,llegar bien perfecto,5
209959,buen producto buen precio,5


In [10]:
corpus = df_reviews["lemmatized_reviews"].tolist()

In [11]:
corpus

['bueno ko pantallar menos mes recibir respuesta fabricante',
 'horrible comprar saber ingls informtico despus hora capaz instalar él',
 'obligar comprar dos unidad llegar solo forma reclamar autentico estafa compreis',
 'entrar descalificar vendedor solo poder decir tras dos mes espera sigo producto contactar amazon reclamar reembolso amazon hacer cargo problema desembolsar dinero dos mes devolver perdido tiempo total palabra tú decidir',
 'llegar tarde co talla equivocado',
 'jams lleg vendedor nunca contacto conmigo pesar intentar él vez',
 'paragua malo calidadda sensacin ir romper abr cutre',
 'devolver él ser triangular agarrir forma cmodo escritura bonito calidadprecio aceptable funcional',
 'esperar despu protestar varios vez ver solucion haceis caso decir reembolso querer auricular',
 'defectuoso apariencia bien producto tercer uso vaso mezclador ms grande dejar funcionar intentar hacer pur requerir fuerza excesivo cuchilla dejar girar pena herramienta principio funcional fina

In [12]:
len(corpus)

209999

## Bag-of-Words with CountVectorizer

In [13]:
model_cv = CountVectorizer()
X_cv = model_cv.fit_transform(corpus)
print(X_cv.shape)
print(model_cv.get_feature_names_out()[:10])


(209999, 60653)
['aa' 'aaa' 'aaaaa' 'aaaaaa' 'aaao' 'aac' 'aad' 'aada' 'aadan' 'aadar']


In [14]:
X_cv

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 2689317 stored elements and shape (209999, 60653)>

## TF-IDF with TfidfVectorizer

In [15]:
model_tfidf = TfidfVectorizer()
X_tfidf = model_tfidf.fit_transform(corpus)
print(X_tfidf.shape)
print(model_tfidf.get_feature_names_out()[:10])

(209999, 60653)
['aa' 'aaa' 'aaaaa' 'aaaaaa' 'aaao' 'aac' 'aad' 'aada' 'aadan' 'aadar']


## LDA

In [16]:
model_lda = LatentDirichletAllocation(n_components=10, random_state=42)
model_lda.fit(X_cv)


,n_components,10
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [18]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Tema %d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [19]:
display_topics(model_lda, model_cv.get_feature_names_out(), 10)

Tema 0:
venir caja él producto bien dar si poder mal ver
Tema 1:
producto vendedor amazon foto poder si ver poner decir mas
Tema 2:
llegar esperar recibir producto mes da dos poder pedido lleg
Tema 3:
calidad él malo cumplir hacer uso ms mes romper funcin
Tema 4:
bien quedar pantalla funda est ver él poner parte cristal
Tema 5:
buen calidad precio producto bien compra cumplir fcil relacin material
Tema 6:
ms pequeo grande él bien si poder demasiado gustar hacer
Tema 7:
perfecto encantar libro talla bien regalo rpido nio hijo encantado
Tema 8:
funcionar él poder si bien cable problema hacer ir vez
Tema 9:
bien luz ms ir batera bastante si gustar est dar


In [20]:
df_reviews["sentiment_bin"] = df_reviews["stars"].apply(lambda x: 1 if x > 3 else 0)

In [21]:
df_reviews.head()

,lemmatized_reviews,stars,sentiment_bin
0,bueno ko pantallar menos mes recibir respuesta...,1,0
1,horrible comprar saber ingls informtico despus...,1,0
2,obligar comprar dos unidad llegar solo forma r...,1,0
3,entrar descalificar vendedor solo poder decir ...,1,0
4,llegar tarde co talla equivocado,1,0


In [22]:
X_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2689317 stored elements and shape (209999, 60653)>

In [23]:
X = X_tfidf
y = df_reviews["sentiment_bin"]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
model_nb_classifier = MultinomialNB()
model_nb_classifier.fit(X_train, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [28]:
# evaluate the model
y_pred = model_nb_classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7899761904761905
              precision    recall  f1-score   support

           0       0.78      0.92      0.84     25204
           1       0.83      0.60      0.70     16796

    accuracy                           0.79     42000
   macro avg       0.80      0.76      0.77     42000
weighted avg       0.80      0.79      0.78     42000



# Save model

In [31]:
path_nb_classifier = MODELS_DIR / "model_nb_classifier.pkl"
print(path_nb_classifier.exists())

False


In [32]:
joblib.dump(model_nb_classifier, path_nb_classifier)
print(path_nb_classifier.exists())

True


# Load Model

In [33]:
my_model_nb_classifier = joblib.load(path_nb_classifier)
print(my_model_nb_classifier)

MultinomialNB()


In [35]:
# Pre-compile regex patterns for efficiency
URL_PATTERN       = re.compile(r'https?://\S+|www\.\S+')
BRACKET_PATTERN   = re.compile(r'\[.*?\]')
HTML_TAG_PATTERN  = re.compile(r'<.*?>')
PUNCT_PATTERN     = re.compile(f"[{re.escape(string.punctuation)}]")
NUM_PATTERN       = re.compile(r'\w*\d\w*')
NON_ASCII_PATTERN = re.compile(r'[^\x00-\x7F]+')
WHITESPACE_PATTERN= re.compile(r'\s+')

def clean_text(text: str) -> str:
    # Ensure input is string
    if not isinstance(text, str):
        text = str(text)

    # 1) Normalize case
    text = text.lower()  # Convert to lowercase

    # 2) Remove unwanted patterns
    text = URL_PATTERN.sub('', text)        # Remove URLs
    text = BRACKET_PATTERN.sub('', text)    # Remove [bracketed] text
    text = HTML_TAG_PATTERN.sub('', text)   # Remove HTML tags
    text = PUNCT_PATTERN.sub('', text)      # Remove punctuation
    text = NUM_PATTERN.sub('', text)        # Remove words with digits
    text = NON_ASCII_PATTERN.sub('', text)  # Remove emojis/non-ASCII

    # 3) Normalize whitespace
    text = WHITESPACE_PATTERN.sub(' ', text)  
    text = text.strip()  # Trim leading/trailing spaces

    return text

In [41]:
# Download the stopwords for Spanish
nltk.download('stopwords', download_dir=DATA_RAW_DIR)




[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jssdev/Dev/Learning/Platzi/platzi-
[nltk_data]     nlp/data/raw...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
# Download the stopwords for Spanish
nltk.download('stopwords', download_dir=DATA_RAW_DIR)

# Configurar NLTK para usar nuestro directorio personalizado
nltk.data.path.insert(0, str(DATA_RAW_DIR))

# Usar las stopwords
stopword_es = set(stopwords.words('spanish'))

# Cargar modelo de spaCy para español
nlp_es = spacy.load('es_core_news_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jssdev/Dev/Learning/Platzi/platzi-
[nltk_data]     nlp/data/raw...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
def clean_with_stopwords_and_lemmatization(text):
    # Procesar el texto usando spaCy
    doc = nlp_es(text)
    # Eliminar stopwords y aplicar lematización
    lemmatized = [token.lemma_ for token in doc if token.text.lower() not in stopword_es]
    # Unir los tokens lematizados y eliminar espacios extra
    return " ".join(lemmatized).strip()

In [46]:
# Ejemplo de nueva reseña
new_review = "Este producto es excelente y superó mis expectativas."

# Preprocesar la reseña usando la función clean + lematización
new_review_clean = clean_text(new_review)
new_review_clean = clean_with_stopwords_and_lemmatization(new_review_clean)

# Transformar el nuevo texto en el mismo espacio vectorial con el vectorizador TF-IDF ya ajustado
new_vector = model_tfidf.transform([new_review_clean])

# Realizar la predicción con el modelo cargado
prediction = my_model_nb_classifier.predict(new_vector)

# Mostrar la predicción (por ejemplo, 1 para positivo, 0 para negativo)
print("Predicción de sentimiento:", prediction[0])

Predicción de sentimiento: 1


In [47]:
# Ejemplo de nueva reseña
new_review = "Lo odio ;)!"

# Preprocesar la reseña usando la función clean + lematización
new_review_clean = clean_text(new_review)
new_review_clean = clean_with_stopwords_and_lemmatization(new_review_clean)

# Transformar el nuevo texto en el mismo espacio vectorial con el vectorizador TF-IDF ya ajustado
new_vector = model_tfidf.transform([new_review_clean])

# Realizar la predicción con el modelo cargado
prediction = my_model_nb_classifier.predict(new_vector)

# Mostrar la predicción (por ejemplo, 1 para positivo, 0 para negativo)
print("Predicción de sentimiento:", prediction[0])

Predicción de sentimiento: 0


In [48]:
# Ejemplo de nueva reseña
new_review = "me encanto?"

# Preprocesar la reseña usando la función clean + lematización
new_review_clean = clean_text(new_review)
new_review_clean = clean_with_stopwords_and_lemmatization(new_review_clean)

# Transformar el nuevo texto en el mismo espacio vectorial con el vectorizador TF-IDF ya ajustado
new_vector = model_tfidf.transform([new_review_clean])

# Realizar la predicción con el modelo cargado
prediction = my_model_nb_classifier.predict(new_vector)

# Mostrar la predicción (por ejemplo, 1 para positivo, 0 para negativo)
print("Predicción de sentimiento:", prediction[0])

Predicción de sentimiento: 1
